<a href="https://colab.research.google.com/github/33quitykubby/Rabbit/blob/main/Rabbit_ML_5_k_means_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# k-meansモデル

k-means(写経）  

skl_kmeans.ipynb  
np_kmeans.ipynb



# マウント

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# 日本時間にする

In [3]:
  !rm /etc/localtime
  !ln -s /usr/share/zoneinfo/Asia/Tokyo /etc/localtime
  !date

Sat Apr 10 16:45:22 JST 2021


# ライブラリのインポート

In [11]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import  confusion_matrix
from sklearn.decomposition import PCA
from scipy import stats

from sklearn import cluster, preprocessing, datasets
from sklearn.cluster import KMeans

from matplotlib import pyplot as plt
import seaborn as sns

#これでplt.show()しなくてよい
%matplotlib inline

import os
import glob

import librosa

import datetime

import gc

In [5]:
#開始時刻
start_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print("start_time=",start_time)

start_time= 2021-04-10 16:45:26.172051+09:00


# 乱数シードの初期化






In [6]:
import os 
import numpy as np
from sklearn import model_selection
import tensorflow as tf
import random

COMMON_SEED = 0
STANDARD_SEED = 0
NP_SEED = 0
TF_SEED = 0 

# np.random.seed(STANDARD_SEED)
# random.seed(NP_SEED)
# tf.random.set_seed(TF_SEED)

def seed_everything():
    random.seed(STANDARD_SEED)
    os.environ['PYTHONHASHSEED'] = str(COMMON_SEED)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    np.random.seed(NP_SEED)
    tf.random.set_seed(TF_SEED)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)


In [7]:
#乱数シード固定

seed_everything()

# パス定義

In [8]:
DATA_PATH = '/content/drive/MyDrive/RabbitChallenge/study_ai_ml_google/data/'
SIZE = 10

# データ準備

ワインデータ

In [16]:
wine = datasets.load_wine()

print(len(wine))
print(wine)


5
{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]]), 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,

In [43]:
X = wine.data

X.shape

print(len(X))

print(X)



178
[[1.423e+01 1.710e+00 2.430e+00 ... 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 ... 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 ... 1.030e+00 3.170e+00 1.185e+03]
 ...
 [1.327e+01 4.280e+00 2.260e+00 ... 5.900e-01 1.560e+00 8.350e+02]
 [1.317e+01 2.590e+00 2.370e+00 ... 6.000e-01 1.620e+00 8.400e+02]
 [1.413e+01 4.100e+00 2.740e+00 ... 6.100e-01 1.600e+00 5.600e+02]]


In [19]:
y=wine.target

y.shape

(178,)

In [20]:
wine.target_names

array(['class_0', 'class_1', 'class_2'], dtype='<U7')

# クラスタリング

## scikitlearn

In [25]:
model = KMeans(n_clusters=3)

labels = model.fit_predict(X)

In [27]:
df = pd.DataFrame({'labels':labels})

type(df)

df

,labels
0,0
1,0
2,0
3,0
4,2
...,...
173,2
174,2
175,2
176,2


In [28]:
def species_label(theta):
  if theta == 0:
    return wine.target_names[0]
  if theta == 1:
    return wine.target_names[1]
  if theta == 2:
    return wine.target_names[2]

In [29]:
df['species'] = [species_label(theta) for theta in wine.target]

pd.crosstab(df['labels'], df['species'])

species,class_0,class_1,class_2
labels,,,
0,46,1,0
1,0,50,19
2,13,20,29


## numpy

In [55]:
# 距離
def distance (x1, x2):
  return np.sum((x1 - x2)**2, axis=1)

_clusters_ = 3
iter_max = 100
# iter_max = 2

# 各クラスタ中心をランダムに初期化
centers = X[np.random.choice(len(X), n_clusters, replace=False)]

for _ in range(iter_max):
  prev_cent = np.copy(centers)

  #データ数×クラスタ数分の領域を確保し初期化
  D = np.zeros((len(X), _clusters_))

  # 各データ点に対して、各クラスタ中心との距離を計算しDに格納
  # Dにはxと3つのクラスタの中心の配列が記録される
  for i, x in enumerate(X):
    D[i] = distance (x, centers)

  # 各データ点に最も距離が近いクラスタを割り当てる
  # argminでDの最も小さいインデックスの配列がcluster_indexに記録される
  cluster_index = np.argmin(D, axis=1)

  print(cluster_index)

  # 各クラスタの中心を計算
  for k in range(n_clusters):
    index_k = cluster_index == k
    centers[k] = np.mean(X[index_k],axis=0)
  
  #収束判定
  if np.allclose(prev_cent, centers):
    print("break")
    break



[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 1 1 0 2 1 1 2 2 2 1 1 2
 2 1 0 1 2 1 0 2 1 1 1 1 1 1 2 1 1 1 1 0 0 2 1 1 2 1 2 1 1 1 2 0 1 1 0 2 1
 0 1 1 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 1 1 1 1 1 1 2 2 1 1 1 1 2 1 1 2 2 1 1
 1 1 1 1 1 2 1 2 1 2 2 1 1 2 1 2 1 1 2 2 2 1 1 1 2 2 2 2 2 1]
[2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 0 0 1 1 1 1 1 2 0 1 2
 2 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 2 1 0 1 0 1 1 1 0 1 0 1 1 0 1 1
 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 2 0 1
 1 1 1 1 0 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1]
[2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 1 1 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 0 1 0 0 1 1 1 1 1 2 0 0 2
 2 0 0 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 1 0 0 2 1 0 1 0 1 1 0 0 1 0 1 1 0 1 1
 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 

In [56]:
df_2 = pd.DataFrame({'labels':cluster_index})


df_2

,labels
0,2
1,2
2,2
3,2
4,1
...,...
173,1
174,1
175,1
176,1


In [57]:
df_2['species'] = [species_label(cluster_index) for cluster_index in wine.target]

pd.crosstab(df_2['labels'], df_2['species'])

species,class_0,class_1,class_2
labels,,,
0,0,50,19
1,13,20,29
2,46,1,0


# 終わり

In [ ]:
#終了時刻
finish_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print("finish_time=",finish_time)
print("total_time=",finish_time-start_time)


finish_time= 2021-04-10 16:10:49.826782+09:00
total_time= 0:00:02.288428
